In [7]:
import pickle
import pandas as pd 
import random
import ast
import numpy as np
import os
import json
import time

In [8]:
chatbot_name = "PeaceWise"
with open('vectorizer.sav', 'rb') as f:
    vectorizer = pickle.load(f)
with open('chatbot_model.pkl', 'rb') as f:
    mnb_model = pickle.load(f)
data = pd.read_csv('data.csv')

EXIT_KEYWORD = "quit"
required = f"Start chatting or enter '{EXIT_KEYWORD}' to exit the program"
starter_prompt = [
    "How can I assist you today with information about mental health?",
    "I'm here to provide you with answers to any questions you have about mental health. Ask away!",
    "Let's talk about mental health and how to keep your mind, body, and spirit peaceful.",
    "I'm your mental health companion. What would you like to know about mental health?",
    "I'm here to provide guidance on understanding and managing mental health. Let's get started.",
    "Let's chat about mental health and how you can maintain a healthy mind and body."
]
confused = [
    "I'm sorry, I didn't quite catch that. Could you please rephrase your question?",
    "Apologies, I'm not sure I follow. Can you clarify or ask a different question?",
    "I'm sorry, I'm not equipped to handle that request. Is there something else I can assist you with?",
    "I'm still learning about mental health and may not have the answer to that question. Is there anything else you'd like to know?"
]

In [9]:
# HELPER METHODS FOR DISPLAYING TEXT
def display_greeting():
    print(f"\nWelcome! My name is PeaceWise, your mental health assistant. What is your name?")

def display_starter_prompt(name):
    print(f"\nHello {name.capitalize()}! {starter_prompt[random.randint(0, len(starter_prompt)-1)]}\n{required}")

def display_confused():
    print(f"\n{confused[random.randint(0, len(confused)-1)]}")

# CHECK IF USERNAME.JSON EXISTS
def does_user_exist(name):
    users = []
    if not os.path.exists("users"):
        os.makedirs("users")
    else:
        for file in os.listdir("users"):
            users.append(file.split(".")[0])

    for user in users:
        if name.lower() == user:
            return True    
    return False

# INITIALIZE USER JSON FILE
def create_user_file(name):
    user_dict = {
        "Name": name,
        "Questions": [],
        "Likes": [],
        "Dislikes": [],
        "Feedback": []
        }
    with open(f'users/{name}.json', 'w') as json_file:
        json.dump(user_dict, json_file, indent=4)

# GIVEN CATEGORY, GET PREVIOUS ANSWER (IF EXISTING)
def get_previous_user_answer(name, category):
    with open(f"users/{name}.json", "r") as file:
        user_dict = json.load(file)
    all_previous_answers = user_dict[category]
    if len(all_previous_answers) == 0:
        return None
    else:
        return all_previous_answers[random.randint(0,len(all_previous_answers)-1)]

# ASK USER A QUESTION ABOUT THEIR LIKES, DISLIKES, OR ASK FOR FEEDBACK
def ask_other_question(name):
    # Questions for each category
    likes = [f"Just wondering {name}, what are your interests?",
            f"Sorry for digress, but can you tell me about your hobbies or interests {name}?",]
    dislikes = [f"Before I forget, I'm curious to know about any activities, foods, or preferences you don't like?",
                "Before we dive deeper into the previous topic, I wanted to ask about your preferences. Are there any things that you'd rather pass on or avoid?"]
    feedback = [f"Sorry to interrupt {name}. Can I get some feedback about my chatbot skills?",
                f"Before we move on, do you mind giving me some feedback about how this conversation is going {name}?"]
    
    # Use random number to select type of question. Display it while referencing previous answer (if available)
    topic_num = random.randint(0,4)
    question_num = random.randint(0,1)
    if topic_num == 0 or topic_num == 1:
        category = "Likes"
        question = likes[question_num]
        previous_answer = get_previous_user_answer(name,category)
        if previous_answer is not None:
            question += f"You previously said one of your {category.lower()} was \"{previous_answer}\". What else would you like to add on?"

    elif topic_num == 2:
        category = "Dislikes"
        question = dislikes[question_num]
        previous_answer = get_previous_user_answer(name,category)
        if previous_answer is not None:
            question += f"You previously said one of your {category.lower()} was \"{previous_answer}\". What else would you like to add on?"
    else:
        category = "Feedback"
        question = feedback[question_num]
        previous_answer = get_previous_user_answer(name,category)
        if previous_answer is not None:
            question += f"You previously said your {category.lower()} was \"{previous_answer}\". What else would you like to add on?"
    
    return question, category

# GIVEN NAME AND QUESTION CATEGORY, WRITE USER-INPUT TO APPROPRIATE FILE IN THE RIGHT PLACE
def write_user_question(name, user_input, category):
    #open name.json
    #Get "Questions" attribute.
    with open(f"users/{name}.json", "r") as file:
        user_dict = json.load(file)
    user_dict[category].append(user_input)

    with open(f'users/{name}.json', 'w') as json_file:
        json.dump(user_dict, json_file, indent=4)
    return

# Function to predict questions based on user input
def predict_question(user_input):
    # Vectorize the user input
    user_input_vec = vectorizer.transform([user_input])

    # Predict the question
    question = mnb_model.predict(user_input_vec)[0]

    # predict proba 
    proba = mnb_model.predict_proba(user_input_vec)
    proba = np.max(proba)
    # print("proba", proba)

    return question, proba

# Function to generate responses based on predicted questions
def generate_response(question):
    # print("predicted question", question)
    response = []
    for index,row in data.iterrows():
        if row['tag'] == question:
            try:
                response = ast.literal_eval(row['responses'])
                break
            except:
                response = row['responses']
    if isinstance(response, list):
        response = response[random.randint(0,len(response)-1)]

    # print(type(response_choices), response_choices)
    return response.replace("Pandora", "PeaceWise").replace(">", "Adithya Viswanathan")

In [10]:
def start_bot():
    display_greeting()
    name = input("> ").lower()
    # check if name exists. if yes, then get user info. if not, create file for them.
    if not does_user_exist(name):
        create_user_file(name)
    display_starter_prompt(name)
    user_input = ""
    while user_input != EXIT_KEYWORD:
        #RANDOMLY ASK QUESTIONS LIKE "ARE YOU FEELING SATISFIED WITH THIS CONVERSTAION" OR "WHAT ARE YOUR INTERESTS?" OR ETC
        qnum = random.randint(1,3)
        # Random question
        if qnum == 3 and user_input != "":
            question, category = ask_other_question(name)
            print("\n",question)
            user_input = input("> ")
            if user_input == EXIT_KEYWORD:
                break
            write_user_question(name, user_input, category)
            print("Thank you for your response!")
            
        # Continue with conversation
        else:
            user_input = input(f"{name} > ")
            print(f"{name} > {user_input}")
            # Predict question
            question, proba = predict_question(user_input)
            if proba < 0.05:
                display_confused()
            else:
                time.sleep(0.5)
                
                # Show response
                response = generate_response(question)

                # print("User:", user_input)
                if user_input == 'quit' or question == "goodbye" or question.lower() == "more-facts156":
                    print(f"{chatbot_name} > {response}")
                    break
                write_user_question(name, user_input, "Questions")

                print(f"{chatbot_name} > {response}")
            

start_bot()


Welcome! My name is PeaceWise, your mental health assistant. What is your name?

Hello Adithya1! Let's talk about mental health and how to keep your mind, body, and spirit peaceful.
Start chatting or enter 'quit' to exit the program
adithya1 > who are you?
PeaceWise > You can call me PeaceWise.
adithya1 > who created you?
PeaceWise > The real question is: Who created you?
adithya1 > I feel scared
PeaceWise > It'll all be okay. This feeling is only momentary.
adithya1 > how do I manage stress?
PeaceWise > Here are some personalized suggestions to help you cope with stress:

 Just wondering adithya1, what are your interests?You previously said one of your likes was "playing with my dog". What else would you like to add on?
Thank you for your response!
adithya1 > i feel stressed
PeaceWise > Take a deep breath and gather your thoughts. Go take a walk if possible. Stay hydrated

 Sorry for digress, but can you tell me about your hobbies or interests adithya1?You previously said one of your